In [4]:
import pandas as pd
import re
df_银行格式 = pd.DataFrame(columns = ['类罪类型',
'反洗钱报告编号',
'场景类型',
'交易ID',
'客户编号',
'交易日期',
'明细来源',
'业务种类',
'业务条线',
'报送业务条线',
'明细序号',
'定活存标志',
'账卡标志',
'客户账号',
'客户名称',
'对公对私标志',
'客户年龄',
'国际卡卡币种',
'币种代码',
'存款种类',
'可售产品编号',
'资金用途代码',
'客户类型代码',
'交易时间',
'交易金额',
'折人民币金额',
'折美元金额',
'借贷标志',
'收付标志',
'交易种类',
'交易方式',
'汇款种类',
'交易渠道种类',
'交易流水号',
'全局事件跟踪号',
'交易柜员号',
'交易代码',
'交易机构',
'交易机构名称',
'交易发生地',
'交易所在国家地区代码',
'交易方向',
'凭证号码',
'凭证种类',
'摘要代码',
'摘要描述',
'账户性质',
'账户类型',
'资金用途',
'开户机构',
'开户机构名称',
'开户机构行政区划',
'反洗钱机构',
'反洗钱机构名称',
'开户日期',
'销户日期',
'上次明细交易日',
'产品种类',
'交易对手系统账号',
'交易对手账号修饰符',
'交易对手账户类型',
'交易对手账户性质',
'交易对手对公对私标志',
'交易对手客户账号',
'交易对手客户编号',
'交易对手客户名称',
'交易对手本外行标识',
'交易对手证件号码',
'交易对手证件类型',
'交易对手证件类型说明',
'交易对手商户编号',
'交易对手商户种类',
'交易对手开户机构',
'交易对手开户机构行政区划',
'交易对手定活存标志',
'交易对手金融机构代码',
'交易对手金融机构名称',
'交易对手金融机构行政区划',
'交易对手金融机构代码网点类型',
'交易对手国籍地区',
'终端信息',
'大额豁免标志',
'老可疑豁免标志',
'新可疑豁免标志',
'代理人姓名',
'代理人证件类型',
'代理人证件类型说明',
'代理人证件号码',
'代理人电话',
'代理人国籍地区',
'是否被代办',
'是否跨境交易',
'结算类型现金转账消费',
'是否跨行交易',
'是否异地交易',
'补录机构',
'账户余额',
'备注1',
'备注2'])

output_path = 'F:\\扬州税侦\\'
all_流水完整 = pd.read_csv(r'F:\扬州税侦\流水完整.csv',dtype = object,encoding = 'utf-8',engine = 'python')
all_流水完整 = all_流水完整.applymap(lambda x: re.sub('nan','',str(x)))
# all_流水完整['交易金额'] = all_流水完整['交易金额'].map(lambda x: float(x))  #交易金额指定浮点型
# all_流水完整['交易余额'] = all_流水完整['交易余额'].map(lambda x: np.NaN if str(x)=='' else x)
# all_流水完整['交易余额'] = all_流水完整['交易余额'].map(lambda x: float(x))

print(all_流水完整.dtypes)

df_代码 = pd.read_excel(r'K:\代码值.xlsx',dtype = 'object',sheet_name = '代码值（全）')
df_币种 = df_代码[df_代码['值域名称'] =='币种']
df_币种英文码 = df_代码[df_代码['值域名称'] =='币种英文码'][['代码编号','代码名称']]
df_币种英文码.columns = ['英文码','代码名称']
df_代码对应 = pd.merge(df_币种,df_币种英文码,how = 'left',on ='代码名称')
df_流水币种 = pd.merge(all_流水完整,df_代码对应,how = 'left',left_on = '交易币种',right_on = '英文码')
print(df_流水币种)


df_银行格式['客户编号'] = all_流水完整['交易账号']
df_银行格式['交易日期'] = all_流水完整['交易时间'].str[0:10]
df_银行格式['客户账号'] = all_流水完整['交易卡号']
df_银行格式['客户名称'] = all_流水完整['交易户名']
df_银行格式['对公对私标志'] = all_流水完整['交易户名'].map(lambda x:'0' if len(x)<=3 else ('0' if '·' in x else ('1' if '公司' in x else '1')))
df_银行格式['币种代码'] = df_流水币种['代码编号']
df_银行格式['交易时间'] = all_流水完整['交易时间'].str.extract(r'(\d{1,2}):\d{1,2}:\d{1,2}')+all_流水完整['交易时间'].str.extract(r'\d{1,2}:(\d{1,2}):\d{1,2}')+all_流水完整['交易时间'].str.extract(r'\d{1,2}:\d{1,2}:(\d{1,2})')
df_银行格式['交易时间'] = pd.to_numeric(df_银行格式['交易时间'],errors='coerce').astype(str)+'.00'
df_银行格式['交易金额'] = all_流水完整['交易金额']
df_银行格式['折人民币金额'] = df_银行格式['交易金额']
df_银行格式['借贷标志'] = all_流水完整['收付标志'].map(lambda x:'C' if x=='进' else 'D')
df_银行格式['交易对手客户账号'] = all_流水完整['交易对手账卡号']
df_银行格式['交易对手客户名称'] = all_流水完整['对手户名']
df_银行格式['交易流水号'] = all_流水完整['交易流水号']
df_银行格式['交易柜员号'] = all_流水完整['交易柜员号']
df_银行格式['交易机构名称'] = all_流水完整['交易网点名称']
df_银行格式['摘要描述'] = all_流水完整['摘要说明']
df_银行格式['资金用途'] = all_流水完整['交易类型']
df_银行格式['终端信息'] = all_流水完整['IP地址']
df_银行格式['账户余额'] = all_流水完整['交易余额']
df_银行格式['备注1'] = all_流水完整['备注']
df_银行格式['反洗钱报告编号'] = '20220323-01'
df_银行格式['场景类型'] = '场景二'

df_银行格式 = df_银行格式.applymap(lambda x: re.sub('nan','',str(x)))
df_银行格式['客户编号'][df_银行格式['客户编号']=='']=df_银行格式['客户账号'][df_银行格式['客户编号']==''] # 用账号填充卡号空白
print(df_银行格式.head(20))

# df_银行格式 = df_银行格式.applymap(lambda x: re.sub('nan',' ',str(x)) )
# df_银行格式[0] = df_银行格式.apply(lambda x: '|@|'.join(x), axis=1)
# pd.DataFrame(df_银行格式[0]+'|@|').to_csv('C:\\Users\\zqy\\Desktop\\'+'P20210257_JS_FXQ_TRAN_20220323_0001.dat',encoding = 'utf-8',header = None, index=False)
# print(df_银行格式[0])



df_3号令 = pd.DataFrame(columns = ['客户编号','反洗钱报告编号','反洗钱可疑交易笔数','金融机构网点名称','金融机构网点所在地区行政区划代码','反洗钱报告对公对私代码','可疑交易特征描述','反洗钱业务规则名称','反洗钱报告最大交易日期','反洗钱报告最小交易日期','反洗钱报告涉及人民币交易金额','反洗钱报告涉及人民币交易笔数','反洗钱传统渠道人民币交易金额','反洗钱电子银行渠道人民币交易金额','反洗钱新兴电子渠道人民币交易金额','反洗钱其他渠道人民币交易金额','反洗钱传统渠道交易笔数','反洗钱电子银行渠道交易笔数','反洗钱新兴电子渠道交易笔数','反洗钱其他渠道交易笔数','可疑交易措施描述','可疑交易报告报送方向代码','反洗钱可疑行为描述','#报告报送状态代码','报送日期','可疑交易报送标志','#批量导入日期','反洗钱交易主体账户个数','反洗钱中心机构编号','反洗钱中心机构中文名称','#备注二(洗钱类型)','报送次数','接续报告参考标志(0-否 1-是)','非可疑行为描述','可疑交易报告疑点分析说明','可疑交易报告疑点分析及排除理由','反洗钱可疑行为描述2','可疑交易报告疑点分析说明2','风险等级'])
df_3号令['客户编号'] = df_银行格式.groupby(['客户编号']).size().reset_index()['客户编号']
df_3号令['反洗钱报告对公对私代码'] = df_银行格式.groupby(['客户编号','对公对私标志']).size().reset_index()['对公对私标志']
df_3号令['反洗钱报告对公对私代码'] = df_3号令['反洗钱报告对公对私代码'].map(lambda x:'2' if x=='1' else '1')
df_3号令['#报告报送状态代码'] = '06'
df_3号令 = df_3号令.applymap(lambda x: re.sub('nan',' ',str(x)) )
# df_3号令[0] = df_3号令.apply(lambda x: '|@|'.join(x), axis=1)
# pd.DataFrame(df_3号令[0]+'|@|').to_csv('C:\\Users\\zqy\\Desktop\\'+'P20210257_JS_FXQ_AML_SSPCS_RPT_20220323_0001.dat',encoding = 'utf-8',header = None, index=False)
# print(df_3号令[0])


交易卡号        object
交易账号        object
查询反馈结果原因    object
交易户名        object
交易证件号码      object
交易时间        object
交易金额        object
交易余额        object
收付标志        object
交易对手账卡号     object
现金标志        object
对手户名        object
对手身份证号      object
对手开户银行      object
摘要说明        object
交易币种        object
交易网点名称      object
交易场所        object
交易发生地       object
交易是否成功      object
传票号         object
IP地址        object
MAC地址       object
对手交易余额      object
交易流水号       object
渠道          object
日志号         object
凭证种类        object
凭证号         object
交易柜员号       object
备注          object
交易类型        object
path        object
序号          object
dtype: object
                       交易卡号                 交易账号 查询反馈结果原因         交易户名  \
0         01600120210017464    01600120210017464       成功  南京健宇达建材有限公司   
1         01600120210017464    01600120210017464       成功  南京健宇达建材有限公司   
2         01600120210017464    01600120210017464       成功  南京健宇达建材有限公司   
3         01600120210017464    0160012021001

   类罪类型      反洗钱报告编号 场景类型 交易ID               客户编号        交易日期 明细来源 业务种类 业务条线  \
0        20220323-01  场景二       01600120210017464  2017-01-06                  
1        20220323-01  场景二       01600120210017464  2017-02-20                  
2        20220323-01  场景二       01600120210017464  2017-02-20                  
3        20220323-01  场景二       01600120210017464  2017-03-21                  
4        20220323-01  场景二       01600120210017464  2017-03-31                  
5        20220323-01  场景二       01600120210017464  2017-03-31                  
6        20220323-01  场景二       01600120210017464  2017-04-19                  
7        20220323-01  场景二       01600120210017464  2017-05-03                  
8        20220323-01  场景二       01600120210017464  2017-05-09                  
9        20220323-01  场景二       01600120210017464  2017-05-09                  
10       20220323-01  场景二       01600120210017464  2017-05-09                  
11       20220323-01  场景二       01600120

In [16]:
print(df_银行格式[df_银行格式[0].str.contains('6013826103001103671')])  # |@|的|会干扰contians，contains会把|当或处理

       类罪类型      反洗钱报告编号 场景类型 交易ID             客户编号        交易日期 明细来源 业务种类  \
223431       20220323-01  场景二          478054037089  2017-01-10             
223432       20220323-01  场景二          478054037089  2017-01-10             
223433       20220323-01  场景二          478054037089  2017-02-09             
223434       20220323-01  场景二          478054037089  2017-03-01             
223435       20220323-01  场景二          478054037089  2017-03-01             
223436       20220323-01  场景二          478054037089  2017-03-01             
223437       20220323-01  场景二          478054037089  2017-03-20             
223438       20220323-01  场景二          478054037089  2017-06-20             
223439       20220323-01  场景二          478054037089  2017-09-20             
223440       20220323-01  场景二          478054037089  2017-10-21             
223441       20220323-01  场景二          478054037089  2017-10-21             
223442       20220323-01  场景二          478054037089  2017-10-21             

In [17]:
df_银行格式[df_银行格式[0].str.contains('6013826103001103671')].to_excel('C:\\Users\\zqy\\Desktop\\'+'1.xlsx')

In [21]:
str(df_银行格式.loc[224328,0])  #这一行导出csv 为什么带“号？

'|@|20220323-01|@|场景二|@||@|478054037089|@|2020-02-23|@||@||@||@||@||@||@||@|6013826103001103671|@|庞建林|@|0|@||@||@|156|@||@||@||@||@|83005.00|@|44.9|@|44.9|@||@|D|@||@||@||@||@||@|61057053|@||@|9881000|@||@||@|中国银行深圳市分行营业部|@||@||@||@||@||@||@|话费充值：实付 44.9 元, 19975036096|@||@||@|话费充值|@||@||@||@||@||@||@||@||@||@||@||@||@||@||@|751061079705|@||@|深圳市年年卡网络科技有限公司|@||@||@||@||@||@||@||@||@||@||@||@||@||@||@||@||@||@||@||@||@||@||@||@||@||@||@||@||@||@||@|619.64|@||@|'

In [5]:
df_银行格式.to_csv('C:\\Users\\zqy\\Desktop\\'+'TRAN_20220325_0001.csv',encoding = 'utf-8', index=False)
df_3号令.to_csv('C:\\Users\\zqy\\Desktop\\'+'AML_SSPCS_RPT_20220325_0001.csv',encoding = 'utf-8', index=False)
print('Done!')

Done!


In [19]:
#df_len = df_银行格式.applymap(lambda x: len(re.sub('nan','',str(x))))
print(df_len.apply(lambda x:max(x),axis = 0))
df_len_max = pd.DataFrame(df_len.apply(lambda x:max(x),axis = 0))
df_len_max.to_excel('C:\\Users\\zqy\\Desktop\\'+'2.xlsx')
print(df_len_max)

类罪类型                0
反洗钱报告编号            11
场景类型                3
交易ID                0
客户编号               26
交易日期               10
明细来源                0
业务种类                0
业务条线                0
报送业务条线              0
明细序号                0
定活存标志               0
账卡标志                0
客户账号               23
客户名称               19
对公对私标志              1
客户年龄                0
国际卡卡币种              0
币种代码                3
存款种类                0
可售产品编号              0
资金用途代码              0
客户类型代码              0
交易时间                9
交易金额               11
折人民币金额             11
折美元金额               0
借贷标志                1
收付标志                0
交易种类                0
                 ... 
交易对手证件类型说明          0
交易对手商户编号            0
交易对手商户种类            0
交易对手开户机构            0
交易对手开户机构行政区划        0
交易对手定活存标志           0
交易对手金融机构代码          0
交易对手金融机构名称          0
交易对手金融机构行政区划        0
交易对手金融机构代码网点类型      0
交易对手国籍地区            0
终端信息               39
大额豁免标志              0
老可疑豁免标志             0
新可疑豁免标志   

In [8]:


df_对公 = df_银行格式[df_银行格式['对公对私标志']=='1']
df_对公 = df_对公.applymap(lambda x: re.sub('nan',' ',str(x)) )
df_对公[0] = df_对公.apply(lambda x: '|@|'.join(x), axis=1)
pd.DataFrame(df_对公[0]+'|@|').to_csv('C:\\Users\\zqy\\Desktop\\'+'P20210257_JS_FXQ_TRAN_20220323_0001.dat',encoding = 'utf-8',header = None, index=False)
print(df_对公[0])


0         |@|20220323-01|@|场景二|@||@|01600120210017464|@|...
1         |@|20220323-01|@|场景二|@||@|01600120210017464|@|...
2         |@|20220323-01|@|场景二|@||@|01600120210017464|@|...
3         |@|20220323-01|@|场景二|@||@|01600120210017464|@|...
4         |@|20220323-01|@|场景二|@||@|01600120210017464|@|...
5         |@|20220323-01|@|场景二|@||@|01600120210017464|@|...
6         |@|20220323-01|@|场景二|@||@|01600120210017464|@|...
7         |@|20220323-01|@|场景二|@||@|01600120210017464|@|...
8         |@|20220323-01|@|场景二|@||@|01600120210017464|@|...
9         |@|20220323-01|@|场景二|@||@|01600120210017464|@|...
10        |@|20220323-01|@|场景二|@||@|01600120210017464|@|...
11        |@|20220323-01|@|场景二|@||@|01600120210017464|@|...
12        |@|20220323-01|@|场景二|@||@|01600120210017464|@|...
13        |@|20220323-01|@|场景二|@||@|01600120210017464|@|...
14        |@|20220323-01|@|场景二|@||@|01600120210017464|@|...
15        |@|20220323-01|@|场景二|@||@|01600120210017464|@|...
16        |@|20220323-01|@|场景二|@||@|0160

In [3]:
df_银行格式['客户编号'] = all_流水完整['交易账号']
df_银行格式['交易日期'] = all_流水完整['交易时间'].str[0:10]
df_银行格式['客户账号'] = all_流水完整['交易卡号']
df_银行格式['客户名称'] = all_流水完整['交易户名']
df_银行格式['对公对私标志'] = all_流水完整['交易户名'].map(lambda x:'0' if len(x)<=3 else ('0' if '·' in x else ('1' if '公司' in x else '')))
df_银行格式['币种代码'] = df_流水币种['代码编号']
df_银行格式['交易时间'] = all_流水完整['交易时间'].str.extract(r'(\d{1,2}):\d{1,2}:\d{1,2}')+all_流水完整['交易时间'].str.extract(r'\d{1,2}:(\d{1,2}):\d{1,2}')+all_流水完整['交易时间'].str.extract(r'\d{1,2}:\d{1,2}:(\d{1,2})')
df_银行格式['交易时间'] = pd.to_numeric(df_银行格式['交易时间'],errors='coerce').astype(str)+'.00'
df_银行格式['交易金额'] = all_流水完整['交易金额'].astype(float).round(2)
df_银行格式['折人民币金额'] = df_银行格式['交易金额']
df_银行格式['借贷标志'] = all_流水完整['收付标志'].map(lambda x:'C' if x=='进' else 'D')
df_银行格式['交易对手客户账号'] = all_流水完整['交易对手账卡号']
df_银行格式['交易对手客户名称'] = all_流水完整['对手户名']
df_银行格式['交易流水号'] = all_流水完整['交易流水号']
df_银行格式['交易柜员号'] = all_流水完整['交易柜员号']
df_银行格式['交易机构名称'] = all_流水完整['交易网点名称']
df_银行格式['摘要描述'] = all_流水完整['摘要说明']
df_银行格式['资金用途'] = all_流水完整['交易类型']
df_银行格式['终端信息'] = all_流水完整['IP地址']
df_银行格式['账户余额'] = all_流水完整['交易余额']
df_银行格式['备注1'] = all_流水完整['备注']
df_银行格式['反洗钱报告编号'] = '20210929-01'
df_银行格式['场景类型'] = '场景一'

df_银行格式 = df_银行格式.applymap(lambda x: re.sub('nan','',str(x)))
df_银行格式['客户编号'][df_银行格式['客户编号']=='']=df_银行格式['客户账号'][df_银行格式['客户编号']==''] # 用账号填充卡号空白
print(df_银行格式.head(20))

   类罪类型      反洗钱报告编号 场景类型 交易ID              客户编号        交易日期 明细来源 业务种类 业务条线  \
0        20210929-01  场景一       0126014430875205  2009-06-21                  
1        20210929-01  场景一       0126014430875205  2009-09-21                  
2        20210929-01  场景一       0126014430875205  2009-12-21                  
3        20210929-01  场景一       0126014430875205  2010-03-21                  
4        20210929-01  场景一       0126014430875205  2010-06-21                  
5        20210929-01  场景一       0126014430875205  2010-09-21                  
6        20210929-01  场景一       0126014430875205  2010-12-21                  
7        20210929-01  场景一       0126014430875205  2011-03-21                  
8        20210929-01  场景一       0126014430875205  2011-06-21                  
9        20210929-01  场景一       0126014430875205  2011-09-21                  
10       20210929-01  场景一       0126014430875205  2011-12-21                  
11       20210929-01  场景一       0126014430875205  20

In [4]:

df_对公 = df_银行格式[df_银行格式['对公对私标志']=='1']
df_对公 = df_对公.applymap(lambda x: re.sub('nan',' ',str(x)) )
df_对公[0] = df_对公.apply(lambda x: '|@|'.join(x), axis=1)
pd.DataFrame(df_对公[0]+'|@|').to_csv('C:\\Users\\zqy\\Desktop\\'+'P20210257_JS_FXQ_TRAN_20211201_0001.dat',encoding = 'utf-8',header = None, index=False)
print(df_对公[0])


df_3号令 = pd.DataFrame(columns = ['客户编号','反洗钱报告编号','反洗钱可疑交易笔数','金融机构网点名称','金融机构网点所在地区行政区划代码','反洗钱报告对公对私代码','可疑交易特征描述','反洗钱业务规则名称','反洗钱报告最大交易日期','反洗钱报告最小交易日期','反洗钱报告涉及人民币交易金额','反洗钱报告涉及人民币交易笔数','反洗钱传统渠道人民币交易金额','反洗钱电子银行渠道人民币交易金额','反洗钱新兴电子渠道人民币交易金额','反洗钱其他渠道人民币交易金额','反洗钱传统渠道交易笔数','反洗钱电子银行渠道交易笔数','反洗钱新兴电子渠道交易笔数','反洗钱其他渠道交易笔数','可疑交易措施描述','可疑交易报告报送方向代码','反洗钱可疑行为描述','#报告报送状态代码','报送日期','可疑交易报送标志','#批量导入日期','反洗钱交易主体账户个数','反洗钱中心机构编号','反洗钱中心机构中文名称','#备注二(洗钱类型)','报送次数','接续报告参考标志(0-否 1-是)','非可疑行为描述','可疑交易报告疑点分析说明','可疑交易报告疑点分析及排除理由','反洗钱可疑行为描述2','可疑交易报告疑点分析说明2','风险等级'])
df_3号令['客户编号'] = df_对公.groupby(['客户编号']).size().reset_index()['客户编号']
df_3号令['反洗钱报告对公对私代码'] = df_对公.groupby(['客户编号','对公对私标志']).size().reset_index()['对公对私标志']
df_3号令['#报告报送状态代码'] = '06'
df_3号令 = df_3号令.applymap(lambda x: re.sub('nan',' ',str(x)) )
df_3号令[0] = df_3号令.apply(lambda x: '|@|'.join(x), axis=1)
pd.DataFrame(df_3号令[0]+'|@|').to_csv('C:\\Users\\zqy\\Desktop\\'+'P20210257_JS_FXQ_AML_SSPCS_RPT_20211201_0001.dat',encoding = 'utf-8',header = None, index=False)
print(df_3号令[0])

1486       |@|20210929-01|@|场景一|@||@|11001069200053004632...
1487       |@|20210929-01|@|场景一|@||@|11001069200053004632...
1488       |@|20210929-01|@|场景一|@||@|11001069200053004632...
1489       |@|20210929-01|@|场景一|@||@|11001069200053004632...
1490       |@|20210929-01|@|场景一|@||@|11001069200053004632...
                                 ...                        
1952911    |@|20210929-01|@|场景一|@||@|79350078801800000937...
1952912    |@|20210929-01|@|场景一|@||@|79350078801800000937...
1952913    |@|20210929-01|@|场景一|@||@|79350078801800000937...
1952914    |@|20210929-01|@|场景一|@||@|79350078801800000937...
1952915    |@|20210929-01|@|场景一|@||@|79350078801800000937...
Name: 0, Length: 57701, dtype: object
0     11001069200053004632|@| |@| |@| |@| |@|1|@| |@...
1     110925283860000|@| |@| |@| |@| |@|1|@| |@| |@|...
2     12084301040037237|@| |@| |@| |@| |@|1|@| |@| |...
3     13050110654800000154|@| |@| |@| |@| |@|1|@| |@...
4     15000096298375|@| |@| |@| |@| |@|1|@| |@| |@| ...
5     15000

In [5]:
df_对公.groupby(['客户编号','客户名称','对公对私标志']).size().reset_index()

,客户编号,客户名称,对公对私标志,0
0,11001069200053004632,北京明德大成教育咨询有限公司,1,2698
1,110925283860000,北京奉善斋餐饮管理有限责任公司,1,269
2,12084301040037237,安徽铁本电子商务有限公司,1,1495
3,13050110654800000154,河北格桑花开物流有限公司,1,30
4,15000096298375,沈阳邦格达润滑油制品有限公司,1,851
5,15000097628521,沈阳多资实业有限公司,1,147
6,15000097888468,沈阳益富达融资租赁有限公司,1,334
7,283065576675,沈阳新经纬金属材料有限公司,1,1223
8,31001509696050003375,上海先荣建筑集团有限公司,1,587
9,342856642480,北京悦悦家商贸有限公司,1,216


In [90]:
(df_对公[0]+'|@|').iloc[0]

' |@|20210929-01|@|场景一|@| |@|01600120210017464|@|2017-01-06|@| |@| |@| |@| |@| |@| |@| |@|01600120210017464|@|南京健宇达建材有限公司|@|1|@| |@| |@|156|@| |@| |@| |@| |@|151640.00|@|20.0|@|20.0|@| |@|D|@| |@| |@| |@| |@| |@|9957A1046300|@| |@|99957030|@| |@| |@|9957|@| |@| |@| |@| |@| |@| |@|网银转账|@| |@| |@||@| |@| |@| |@| |@| |@| |@| |@| |@| |@| |@| |@| |@| |@| |@|6223050311413648|@| |@|陈国丽|@| |@| |@| |@| |@| |@| |@| |@| |@| |@| |@| |@| |@| |@| |@||@| |@| |@| |@| |@| |@| |@| |@| |@| |@| |@| |@| |@| |@| |@| |@|6.38|@|还款|@| |@|'

In [7]:
(df_3号令[0]+'|@|').iloc[0]

'11001069200053004632|@| |@| |@| |@| |@|1|@| |@| |@| |@| |@| |@| |@| |@| |@| |@| |@| |@| |@| |@| |@| |@| |@| |@|06|@| |@| |@| |@| |@| |@| |@| |@| |@| |@| |@| |@| |@| |@| |@| |@|'

In [6]:
import pandas as pd
import re

output_path = 'I:\\其他洗钱类清洗\\南通清洗\\'
all_流水完整 = pd.read_csv(output_path +"流水完整.csv",dtype = 'object')
all_流水完整 = all_流水完整.applymap(lambda x: re.sub('nan','',str(x)))
df_对手 = all_流水完整.groupby(['交易对手账卡号','对手户名']).size().reset_index()

df_收单代码 =  pd.read_excel(r'C:\Users\zqy\Documents\火模\示例模型\示例数据\收单机构.xlsx',dtype = 'object')
df_挂牌三方 =  pd.read_excel(r'C:\Users\zqy\Documents\火模\示例模型\示例数据\中国人民银行挂牌第三方支付机构.xlsx',dtype = 'object')

df_匹配 = pd.DataFrame()
for str_regex in df_收单代码['正则匹配'].tolist():
    df_筛选 = df_对手[df_对手['交易对手账卡号'].str.contains(str_regex)]
    df_匹配 = df_匹配.append(df_筛选)
    print(str_regex)
    print(df_筛选)
df_交易对手 = df_对手[~df_对手['交易对手账卡号'].isin(df_匹配['交易对手账卡号'])]

df_筛选 = df_交易对手[df_交易对手['交易对手账卡号'].str.contains('^2[0-9]{8}$')]
df_匹配 = df_匹配.append(df_筛选)
print(df_筛选)
df_交易对手 = df_交易对手[~df_交易对手['交易对手账卡号'].str.contains('^2[0-9]{8}')]
            
for str_regex in df_挂牌三方['简称'].tolist():
    df_筛选 = df_交易对手[df_交易对手['对手户名'].str.contains(str_regex)]
    df_匹配 = df_匹配.append(df_筛选)
    print(str_regex)
    print(df_筛选)
df_交易对手 = df_交易对手[~df_交易对手['交易对手账卡号'].isin(df_匹配['交易对手账卡号'])]
df_匹配.to_excel('C:\\Users\\zqy\\Desktop\\新建文件夹\\干扰社群图计算的对手账户 库\\干扰账户清单.xlsx')
print('Done!')

^801[0-9]{12}$
Empty DataFrame
Columns: [交易对手账卡号, 对手户名, 0]
Index: []
^802[0-9]{12}$
                交易对手账卡号                  对手户名   0
322169  802080290000015         中国铁路总公司资金清算中心   1
322170  802080290000015            中国铁路网络有限公司   4
322171  802110060510540     （特约）沈阳金融商贸开发区普罗米斯   2
322172  802110060510540  （特约）沈阳金融商贸开发区普罗米斯小额贷   5
322173  802140059330502            （特约）晋商消费金融   2
...                 ...                   ...  ..
322230  802520048160505     （特约）iCloud由云上贵州运营  29
322231  802620200429151                   韩同鹏   1
322232  802750200376491        青岛亿承顺发国际商贸有限公司   1
322233  802780200299234           青岛盛泓达建材有限公司   1
322234  802920260510514     （特约）沈阳金融商贸开发区普罗米斯   4

[66 rows x 3 columns]
^8031000[0-9]{8}$
Empty DataFrame
Columns: [交易对手账卡号, 对手户名, 0]
Index: []
^804[0-9]{12}$
Empty DataFrame
Columns: [交易对手账卡号, 对手户名, 0]
Index: []
^806[0-9]{12}$
Empty DataFrame
Columns: [交易对手账卡号, 对手户名, 0]
Index: []
^808[0-9]{12}$
Empty DataFrame
Columns: [交易对手账卡号, 对手户名, 0]
Index: []
^808[0-9]{12}$

^8192900[0-9]{8}$
                交易对手账卡号      对手户名  0
322856  819290049003932  全国自助终端缴费  1
322857  819290049006393  全国自助缴费终端  1
^820[0-9]{12}$
Empty DataFrame
Columns: [交易对手账卡号, 对手户名, 0]
Index: []
^821[0-9]{12}$
                交易对手账卡号              对手户名   0
322862  821100060120159  （特约）北京闪银奇异科技有限公司   2
322864  821131155118441      冠松丰田汽车销售有限公司   1
322865  821140970110045       山西省五台山栖贤阁宾馆   1
322868  821150270110013   包头市光辉假日酒店有限责任公司   1
322869  821150755117079           <0!=(.;   1
...                 ...               ...  ..
322939  821640060121005   （特约）中国光大银行信用卡中心   4
322940  821640060121006        （特约）民生（信还）   1
322941  821640060121007     （特约）平安银行信用卡中心  17
322942  821640060121012  （特约）中信百信银行股份有限公司   2
322943  821640060121013  （特约）四川新网银行股份有限公司   2

[72 rows x 3 columns]
^822[0-9]{12}$
                交易对手账卡号                 对手户名   0
322948  822100049000081            拉卡拉支付有限公司   1
322949  822100050943932               北京富源珠宝   1
322955  822290058126330       上海华云海鸥餐饮管理有限公司   1
3

^835[0-9]{12}$
                交易对手账卡号                  对手户名  0
323612  835440145110005    （特约）北京金色世纪商旅网络科技公司  2
323613  835440145110056     （特约）南京银燕航空旅游(互联网)  3
323614  835440145110074       （特约）深圳腾邦国际(互联网)  1
323615  835440145110091  （特约）北京至尊荟科技有限公司(互联网)  1
323616  835440145110098   （特约）北京讯奇连城商务公司(互联网)  1
323617  835440148168001           （特约）银联在线支付D  1
323618  835440160510001                  易联支付  3
^836[0-9]{12}$
                交易对手账卡号              对手户名  0
323619  836010454119045  上海市普陀区华信基业日用百货商行  1
323620  836010854116968        上海市盛昌布料批发部  1
323621  836011054111410        北京市房山区鑫发超市  1
323622  836011554116262     天津市荣兴顺预包装食品超市  1
323623  836100050450647      北京苹果星辰销售电子产品  1
323624  836102658120006       北京市品味佳缘餐饮服务  1
323625  836103952510065    北京多柔美家建筑装饰有限公司  5
323626  836290659210013         上海强捷商贸经营部  9
323627  836584054116674        深圳市宝安区顶盛超市  1
^837[0-9]{12}$
                交易对手账卡号                 对手户名   0
323628  837100060125007     （特约）北京闪银奇异科技有限公司  41
323629  8372900

^103[0-9]{12}$
              交易对手账卡号                                      对手户名   0
2187  103053258120498                                城阳区鑫本盛海鲜酒楼   2
2189  103100055417634                             北京蓝图纷飞加油站有限公司   1
2190  103100070113062                        船舶重工大厦有限公司北京船舶重工酒店   1
2191  103100070117111                            北京怡江海庭酒店管理有限公司   1
2193  103110392223327                               拉卡拉河北银行个贷还款   1
2194  103114445240493                                       张明明   2
2195  103131058124178                                固安县固安镇龙江饭店   1
2196  103134054110030                               惠友购物广场容城88店   2
2202  103320192221856                            江苏省公安厅交通巡逻警察总队   1
2203  103330148140003                         浙江开心果网络科技有限公司（移动）   1
2204  103360551398125                                 新余市个性百变鞋店   2
2205  103360556416176                          新余市个体户华冬秀（若贝儿童装）   1
2206  103371393990431                          临沂市公安局交通警察支队莒南大队   7
2207  103411454110900            

^308[0-9]{12}$
               交易对手账卡号                 对手户名   0
13067  308110053110105             （特约）百度团购   1
13068  308110053110105             （特约）百度服务  17
13069  308310060124267                SHCMB   1
13070  308440149370006         广州易联商业服务有限公司   1
13071  308440149370006             易联支付有限公司   1
13072  308440149580021         快钱支付清算信息有限公司   1
13073  308440394980003          （特约）信用卡对账还款  23
13074  308440394980008      （特约）掌上生活他行信用卡还款  66
13075  308440650219112         乐从家私城博领家具经营部   1
13076  308441315200021       惠州市康宏房地产开发有限公司   1
13077  308441356990038         惠州市惠城区哎莫特服装店   1
13078  308999853110223  （特约）招行手机银行一网通（网上商城）   2
13079  308999860120007        （特约）招行手机银行一网通  60
^309[0-9]{12}$
               交易对手账卡号                                    对手户名     0
13080  309003250450012                    CHNSHSHH北京诚通时代商贸有限公司     1
13081  309003252717002                 CHNSHSHH北京百利盛昌旧机动车经纪有限公     1
13082  309003350650011                 CHNSHSHH惠州雍逸园颂兰阁世通冷气设备工     1
13083  309003350650057

...

拉卡拉
                     交易对手账卡号                对手户名   0
296                                        拉卡拉商户   2
10687    1111111111111111111         拉卡拉支付股份有限公司   3
13501   33001618835059666666      拉卡拉支付有限公司客户备付金   1
14523    4000051319100073215      拉卡拉支付有限公司客户备付金   6
14851      41025901941001356         拉卡拉支付股份有限公司   5
15180     431210100100026799           拉卡拉终端有限公司   1
17343   44001400115053005458    拉卡拉支付有限公司（客户备付金）   1
17344   44001400115053005458  拉卡拉支付股份有限公司（客户备付金）  17
328889          991100000259        拉卡拉支付有限公司备付金  23
328975     99999999999999999                 拉卡拉   9
付费通
Empty DataFrame
Columns: [交易对手账卡号, 对手户名, 0]
Index: []
平安付
                           交易对手账卡号              对手户名    0
898                                  （特约）平安付T0（资金结算）    1
899                                  （特约）平安付科技服务有限公司    1
1096               000400040026041       平安付科技服务有限公司   57
1404              0258014170000361  平安付科技服务有限公司客户备付金    3
1732           1001170319006734623  平安付科技服务有限公司客户备付金   39
173

海科融通
                交易对手账卡号              对手户名    0
134                       北京海科融通支付服务股份有限公    1
14892   410350248160184  (特约)海科融通T0（资金结算）    3
328890     991100000656   北京海科融通支付服务股份有限公  173
328891     991100000656  北京海科融通支付服务股份有限公司   11
中联信
Empty DataFrame
Columns: [交易对手账卡号, 对手户名, 0]
Index: []
腾付通
Empty DataFrame
Columns: [交易对手账卡号, 对手户名, 0]
Index: []
东方付通
Empty DataFrame
Columns: [交易对手账卡号, 对手户名, 0]
Index: []
易智付
                     交易对手账卡号                  对手户名    0
413                                      易智付科技有限公司  156
414                               易智付科技（北京）有限公司备付金    4
924                               （特约）易智付（商户清结算）T0    9
1077        0000300000000485       易智付科技（北京）有限公司备付    3
1092         000300000000485      易智付科技（北京）有限公司备付金    2
1319     0200053429067081260    易智付科技（北京）有限公司客户备付金  510
10530   11001085300059618471    易智付科技（北京）有限公司客户备付金   37
10711      11250801040014647         易智付科技（北京）有限公司  646
10712      11250801040014647    易智付科技（北京）有限公司客户备付金  190
10713      1125080194

上海瀚银
                    交易对手账卡号               对手户名   0
51                             上海瀚银信息技术有限公司备付金  54
1467      03301800040017048       上海瀚银信息技术有限公司  96
1750    1001222329024868841  上海瀚银信息技术有限公司客户备付金   1
328919         991290001042       上海瀚银信息技术有限公司   1
328920         991290001042    上海瀚银信息技术有限公司备付金  17
银视通
Empty DataFrame
Columns: [交易对手账卡号, 对手户名, 0]
Index: []
东方汇融
Empty DataFrame
Columns: [交易对手账卡号, 对手户名, 0]
Index: []
融宝支付
                    交易对手账卡号               对手户名   0
976                            （特约）融宝支付（商户清结算）   1
1460    0302098029100050665  天津融宝支付网络有限公司客户备付金  53
10745  12001615300052526174  天津融宝支付网络有限公司客户备付金   8
易付宝
                     交易对手账卡号                  对手户名   0
565                                       苏宁易付宝-暂无   2
975                               （特约）苏宁易付宝（商户清结算）  12
1849       10102401040005164  南京苏宁易付宝网络科技有限公司客户备付金  11
11066        125904572110701       南京苏宁易付宝网络科技有限公司   1
13433   32001881436052507354   南京苏宁易付宝网络科技有限公司客户备付   1
13434   32001881436052507354 

银博盛世
Empty DataFrame
Columns: [交易对手账卡号, 对手户名, 0]
Index: []
银通支付
Empty DataFrame
Columns: [交易对手账卡号, 对手户名, 0]
Index: []
银通支付
Empty DataFrame
Columns: [交易对手账卡号, 对手户名, 0]
Index: []
高汇通
Empty DataFrame
Columns: [交易对手账卡号, 对手户名, 0]
Index: []
银信联
Empty DataFrame
Columns: [交易对手账卡号, 对手户名, 0]
Index: []
广聚福
Empty DataFrame
Columns: [交易对手账卡号, 对手户名, 0]
Index: []
北京商银
Empty DataFrame
Columns: [交易对手账卡号, 对手户名, 0]
Index: []
国旅（北京）信息科技有限公司
Empty DataFrame
Columns: [交易对手账卡号, 对手户名, 0]
Index: []
重庆城市通卡支付有限责任公司
Empty DataFrame
Columns: [交易对手账卡号, 对手户名, 0]
Index: []
重庆市公众城市一卡通有限责任公司
Empty DataFrame
Columns: [交易对手账卡号, 对手户名, 0]
Index: []
重庆千礼科技有限公司
Empty DataFrame
Columns: [交易对手账卡号, 对手户名, 0]
Index: []
御嘉支付
Empty DataFrame
Columns: [交易对手账卡号, 对手户名, 0]
Index: []
山西易联
Empty DataFrame
Columns: [交易对手账卡号, 对手户名, 0]
Index: []
吉林省通卡支付股份有限公司
Empty DataFrame
Columns: [交易对手账卡号, 对手户名, 0]
Index: []
明生商盟
Empty DataFrame
Columns: [交易对手账卡号, 对手户名, 0]
Index: []
银付通
Empty DataFrame
Columns: [交易对手账卡号, 对手户名, 0]
Index: []
福建一卡通网络有限责任公司

商物通
                     交易对手账卡号                对手户名   0
17347   44001490505053005113  广州商物通网络科技有限公司客户备付金  55
328931          991581002580      广州商物通网络科技有限公司备   1
328932          991581002580     广州商物通网络科技有限公司备付   3
328933          991581002580    广州商物通网络科技有限公司备付金   5
汇聚支付
                            交易对手账卡号                    对手户名     0
244                                          广州市汇聚支付电子科技有限公     1
245                                         广州市汇聚支付电子科技有限公司   106
453                                                    汇聚支付   120
936                                        （特约）汇聚支付T1（资金结算）    37
937                                        （特约）汇聚支付（实时清算18）    17
938                                           （特约）汇聚支付（清算款）    24
1059               0000300000000165          广州市汇聚支付电子科技有限公    25
1060               0000300000000165         广州市汇聚支付电子科技有限公司   213
1061               0000300000000165      广州市汇聚支付电子科技有限公司备付金     1
1183    010860001156313499000000025      广州市汇聚支付电子科技有限公司备付金     2


In [9]:
import pandas as pd
import re

output_path = 'I:\\网赌类数据清洗\\舒城清洗\\'
all_流水完整 = pd.read_csv(output_path +"流水完整.csv",dtype = 'object')
all_流水完整 = all_流水完整.applymap(lambda x: re.sub('nan','',str(x)))
df_对手 = all_流水完整.groupby(['交易对手账卡号','对手户名']).size().reset_index()

df_收单代码 =  pd.read_excel(r'C:\Users\zqy\Documents\火模\示例模型\示例数据\收单机构.xlsx',dtype = 'object')
df_挂牌三方 =  pd.read_excel(r'C:\Users\zqy\Documents\火模\示例模型\示例数据\中国人民银行挂牌第三方支付机构.xlsx',dtype = 'object')

df_匹配 = pd.DataFrame()
for str_regex in df_收单代码['正则匹配'].tolist():
    df_筛选 = df_对手[df_对手['交易对手账卡号'].str.contains(str_regex)]
    df_匹配 = df_匹配.append(df_筛选)
    print(str_regex)
    print(df_筛选)
df_交易对手 = df_对手[~df_对手['交易对手账卡号'].isin(df_匹配['交易对手账卡号'])]

df_筛选 = df_交易对手[df_交易对手['交易对手账卡号'].str.contains('^2[0-9]{8}$')]
df_匹配 = df_匹配.append(df_筛选)
print(df_筛选)
df_交易对手 = df_交易对手[~df_交易对手['交易对手账卡号'].str.contains('^2[0-9]{8}')]
            
for str_regex in df_挂牌三方['简称'].tolist():
    df_筛选 = df_交易对手[df_交易对手['对手户名'].str.contains(str_regex)]
    df_匹配 = df_匹配.append(df_筛选)
    print(str_regex)
    print(df_筛选)
df_交易对手 = df_交易对手[~df_交易对手['交易对手账卡号'].isin(df_匹配['交易对手账卡号'])]
df_匹配.to_excel('C:\\Users\\zqy\\Desktop\\新建文件夹\\干扰社群图计算的对手账户 库\\干扰账户清单.xlsx')
print('Done!')

^801[0-9]{12}$
Empty DataFrame
Columns: [交易对手账卡号, 对手户名, 0]
Index: []
^802[0-9]{12}$
                交易对手账卡号                 对手户名  0
392538  802110053980902             （特约）美团小贷  6
392539  802301048140002               欧飞(电信)  2
392540  802310053980538   （特约）瓜子融资租赁(中国)有限公司  1
392541  802310053980628       （特约）徽商银行股份有限公司  3
392542  802310059620516  （特约）上海华程西南国际旅行社有限公司  4
392543  802310060510537         （特约）银行股份有限公司  1
392544  802310063000562           （特约）中原消费金融  4
392545  802310063000562             （特约）美团小贷  5
392546  802310063000562         （特约）银行股份有限公司  4
392547  802510200984144                  韩军苹  3
392548  802520048160505   （特约）iCloud 由云上贵州运营  3
^8031000[0-9]{8}$
Empty DataFrame
Columns: [交易对手账卡号, 对手户名, 0]
Index: []
^804[0-9]{12}$
Empty DataFrame
Columns: [交易对手账卡号, 对手户名, 0]
Index: []
^806[0-9]{12}$
Empty DataFrame
Columns: [交易对手账卡号, 对手户名, 0]
Index: []
^808[0-9]{12}$
                交易对手账卡号        对手户名  0
392553  808080211306153  招联消费金融有限公司  1
^808[0-9]{12}$
                交易对手账卡

^826[0-9]{12}$
                交易对手账卡号             对手户名   0
392741  826440372990003     银盛支付服务股份有限公司  19
392742  826440372990003  银盛支付服务股份有限公司备付金   7
392743  826440372990003     银盛科技服务集团有限公司  75
392744  826538170110011   湖北金皇国际假日酒店有限公司   1
392745  826701553110002     贵州福爱企业管理有限公司   1
^827[0-9]{12}$
Empty DataFrame
Columns: [交易对手账卡号, 对手户名, 0]
Index: []
^828[0-9]{12}$
Empty DataFrame
Columns: [交易对手账卡号, 对手户名, 0]
Index: []
^829[0-9]{12}$
                交易对手账卡号     对手户名   0
392746  829310072995000  （特约）巨优惠  13
^831[0-9]{12}$
                交易对手账卡号      对手户名   0
392751  831011000007362  易生支付有限公司  19
392752  831310089990004  易生商户资金清算   1
^833[0-9]{12}$
                交易对手账卡号                        对手户名  0
392753  833012457350003  bill.itunes.com MN3SL7GVQL  1
392754  833012457350003  bill.itunes.com MQNTD4JNH9  1
392755  833012457350003  bill.itunes.com MT4SS89W93  1
392756  833012457350003  bill.itunes.com MT4SSGVSSQ  1
392757  833012457350003  bill.itunes.com MT4SWFM4KH  1
...           

^103[0-9]{12}$
              交易对手账卡号                对手户名   0
2428  103007364435034      二维码消费（网商银行APP）   1
2429  103008190796521      二维码消费（网商银行APP）   1
2430  103330460120005  贷款还款 （特约）微粒贷（信贷单笔还  11
2431  103341453110225       巢湖安德利购物中心有限公司   1
^104[0-9]{12}$
              交易对手账卡号                对手户名   0
2432  104000094980008  （特约）中国银行银行卡中心信用卡还款   5
2434  104053293110002                       4
2435  104100548991020            京东商城平台商户   1
2437  104110082202417             支付宝快捷支付  87
2438  104110545111286                去哪儿网   2
2440  104410956512343        濮阳市玉兰花园雅茜服装店   1
2441  104450183980069              广西慈善总会   1
2443  104481445111967                 财付通   2
^105[0-9]{12}$
              交易对手账卡号                   对手户名   0
2460  105000048140248          深圳市年年卡网络科技有限公   2
2461  105000060120028                 龙支付清算户   1
2462  105000073991101       新浪支付-无锡源石云科技有限公司  75
2463  105000073991101   新浪支付-深圳市小赢信息技术有限责任公司   9
2464  105000073992925  汇潮支付有限公司-海南云展网络科技有限公司   1
...              

^4145210[0-9]{8}$
Empty DataFrame
Columns: [交易对手账卡号, 对手户名, 0]
Index: []
^4145310[0-9]{8}$
Empty DataFrame
Columns: [交易对手账卡号, 对手户名, 0]
Index: []
^4154560[0-9]{8}$
Empty DataFrame
Columns: [交易对手账卡号, 对手户名, 0]
Index: []
^4162640[0-9]{8}$
Empty DataFrame
Columns: [交易对手账卡号, 对手户名, 0]
Index: []
^417[0-9]{12}$
Empty DataFrame
Columns: [交易对手账卡号, 对手户名, 0]
Index: []
^4171100[0-9]{8}$
Empty DataFrame
Columns: [交易对手账卡号, 对手户名, 0]
Index: []
^4172210[0-9]{8}$
Empty DataFrame
Columns: [交易对手账卡号, 对手户名, 0]
Index: []
^4184930[0-9]{8}$
Empty DataFrame
Columns: [交易对手账卡号, 对手户名, 0]
Index: []
^4185050[0-9]{8}$
Empty DataFrame
Columns: [交易对手账卡号, 对手户名, 0]
Index: []
^4192310[0-9]{8}$
Empty DataFrame
Columns: [交易对手账卡号, 对手户名, 0]
Index: []
^4202220[0-9]{8}$
Empty DataFrame
Columns: [交易对手账卡号, 对手户名, 0]
Index: []
^4221200[0-9]{8}$
Empty DataFrame
Columns: [交易对手账卡号, 对手户名, 0]
Index: []
^4233310[0-9]{8}$
Empty DataFrame
Columns: [交易对手账卡号, 对手户名, 0]
Index: []
^4243010[0-9]{8}$
Empty DataFrame
Columns: [交易对手账卡号, 对手户名, 0]
Index

^5197117[0-9]{8}$
Empty DataFrame
Columns: [交易对手账卡号, 对手户名, 0]
Index: []
^5207020[0-9]{8}$
Empty DataFrame
Columns: [交易对手账卡号, 对手户名, 0]
Index: []
^5213000[0-9]{8}$
Empty DataFrame
Columns: [交易对手账卡号, 对手户名, 0]
Index: []
^5238330[0-9]{8}$
Empty DataFrame
Columns: [交易对手账卡号, 对手户名, 0]
Index: []
^5238333[0-9]{8}$
Empty DataFrame
Columns: [交易对手账卡号, 对手户名, 0]
Index: []
^5247410[0-9]{8}$
Empty DataFrame
Columns: [交易对手账卡号, 对手户名, 0]
Index: []
^5253450[0-9]{8}$
Empty DataFrame
Columns: [交易对手账卡号, 对手户名, 0]
Index: []
^5264330[0-9]{8}$
Empty DataFrame
Columns: [交易对手账卡号, 对手户名, 0]
Index: []
^5274550[0-9]{8}$
Empty DataFrame
Columns: [交易对手账卡号, 对手户名, 0]
Index: []
^5284630[0-9]{8}$
Empty DataFrame
Columns: [交易对手账卡号, 对手户名, 0]
Index: []
^5295280[0-9]{8}$
Empty DataFrame
Columns: [交易对手账卡号, 对手户名, 0]
Index: []
^5303380[0-9]{8}$
Empty DataFrame
Columns: [交易对手账卡号, 对手户名, 0]
Index: []
^5311930[0-9]{8}$
Empty DataFrame
Columns: [交易对手账卡号, 对手户名, 0]
Index: []
^5326550[0-9]{8}$
Empty DataFrame
Columns: [交易对手账卡号, 对手户名, 0]
In

^4448800[0-9]{8}$
Empty DataFrame
Columns: [交易对手账卡号, 对手户名, 0]
Index: []
^4449044[0-9]{8}$
Empty DataFrame
Columns: [交易对手账卡号, 对手户名, 0]
Index: []
^4452400[0-9]{8}$
Empty DataFrame
Columns: [交易对手账卡号, 对手户名, 0]
Index: []
^4452402[0-9]{8}$
Empty DataFrame
Columns: [交易对手账卡号, 对手户名, 0]
Index: []
^4468700[0-9]{8}$
Empty DataFrame
Columns: [交易对手账卡号, 对手户名, 0]
Index: []
^4473600[0-9]{8}$
Empty DataFrame
Columns: [交易对手账卡号, 对手户名, 0]
Index: []
^4486400[0-9]{8}$
Empty DataFrame
Columns: [交易对手账卡号, 对手户名, 0]
Index: []
^4498500[0-9]{8}$
Empty DataFrame
Columns: [交易对手账卡号, 对手户名, 0]
Index: []
^4505800[0-9]{8}$
Empty DataFrame
Columns: [交易对手账卡号, 对手户名, 0]
Index: []
^4511900[0-9]{8}$
Empty DataFrame
Columns: [交易对手账卡号, 对手户名, 0]
Index: []
^4526500[0-9]{8}$
Empty DataFrame
Columns: [交易对手账卡号, 对手户名, 0]
Index: []
^4538200[0-9]{8}$
Empty DataFrame
Columns: [交易对手账卡号, 对手户名, 0]
Index: []
^4542200[0-9]{8}$
Empty DataFrame
Columns: [交易对手账卡号, 对手户名, 0]
Index: []
^4551600[0-9]{8}$
Empty DataFrame
Columns: [交易对手账卡号, 对手户名, 0]
In

^5535813[0-9]{8}$
Empty DataFrame
Columns: [交易对手账卡号, 对手户名, 0]
Index: []
^5572210[0-9]{8}$
Empty DataFrame
Columns: [交易对手账卡号, 对手户名, 0]
Index: []
^5584510[0-9]{8}$
Empty DataFrame
Columns: [交易对手账卡号, 对手户名, 0]
Index: []
^5584513[0-9]{8}$
Empty DataFrame
Columns: [交易对手账卡号, 对手户名, 0]
Index: []
^5584630[0-9]{8}$
Empty DataFrame
Columns: [交易对手账卡号, 对手户名, 0]
Index: []
^5584631[0-9]{8}$
Empty DataFrame
Columns: [交易对手账卡号, 对手户名, 0]
Index: []
^5584633[0-9]{8}$
Empty DataFrame
Columns: [交易对手账卡号, 对手户名, 0]
Index: []
^5584710[0-9]{8}$
Empty DataFrame
Columns: [交易对手账卡号, 对手户名, 0]
Index: []
^5584712[0-9]{8}$
Empty DataFrame
Columns: [交易对手账卡号, 对手户名, 0]
Index: []
^5584713[0-9]{8}$
Empty DataFrame
Columns: [交易对手账卡号, 对手户名, 0]
Index: []
^5584715[0-9]{8}$
Empty DataFrame
Columns: [交易对手账卡号, 对手户名, 0]
Index: []
^5584770[0-9]{8}$
Empty DataFrame
Columns: [交易对手账卡号, 对手户名, 0]
Index: []
^5585511[0-9]{8}$
Empty DataFrame
Columns: [交易对手账卡号, 对手户名, 0]
Index: []
^5585514[0-9]{8}$
Empty DataFrame
Columns: [交易对手账卡号, 对手户名, 0]
In

拉卡拉
                交易对手账卡号         对手户名    0
275                      拉卡拉支付股份有限公司  134
10226      300000000222  拉卡拉支付股份有限公司    1
393849  991100000259022  拉卡拉支付股份有限公司  173
付费通
    交易对手账卡号 对手户名  0
124          付费通  2
平安付
                 交易对手账卡号            对手户名  0
1585    0000300000000169  平安付科技服务有限公司备付金  9
10088          24*****39     平安付科技服务有限公司  1
393912   991584000274840  平安付科技服务有限公司备付金  1
上海银联
                交易对手账卡号            对手户名  0
393879  991290000283909  上海银联电子支付服务有限公司  1
连连银通
                  交易对手账卡号             对手户名   0
1366                          连连银通电子支付有限公司  21
1573      000000048679202     连连银通电子支付有限公司   3
1582     0000300000000161  连连银通电子支付有限公司备付金   3
393895    991331000295310     连连银通电子支付有限公司   3
393896    991331000295310  连连银通电子支付有限公司备付金   8
393977  99999999999999999             连连银通  13
联动优势
             交易对手账卡号             对手户名   0
1002                       联动优势电子商务有限   2
393850  991100000306  联动优势电子商务有限公司备付金  35
成都摩宝
Empty DataFrame
Columns: [交易对手账卡号, 对手户名, 0]

一九付支付
Empty DataFrame
Columns: [交易对手账卡号, 对手户名, 0]
Index: []
数码视讯
Empty DataFrame
Columns: [交易对手账卡号, 对手户名, 0]
Index: []
汇元银通
Empty DataFrame
Columns: [交易对手账卡号, 对手户名, 0]
Index: []
随行付
                 交易对手账卡号          对手户名    0
1418                      随行付支付有限公司备付金  128
1596    0000300000000256  随行付支付有限公司备付金  880
393859   991100001171032     随行付支付有限公司    7
393860   991100001171032  随行付支付有限公司备付金    4
网易宝
Empty DataFrame
Columns: [交易对手账卡号, 对手户名, 0]
Index: []
唯品会
                交易对手账卡号                对手户名  0
203                      唯品会支付-广州唯品会电子商务有限公  1
1138                                财付通-唯品会  1
1180                      财付通-广州唯品会电子商务有限公司  1
393216  937310053110017                 唯品会  4
浙江航天电子信息产业有限公司
Empty DataFrame
Columns: [交易对手账卡号, 对手户名, 0]
Index: []
甬易电子支付
Empty DataFrame
Columns: [交易对手账卡号, 对手户名, 0]
Index: []
国通星驿
                交易对手账卡号            对手户名    0
955                      福建国通星驿网络科技有限公司   88
393900  991391001221910  福建国通星驿网络科技有限公司  817
九派天下
Empty DataFrame
Columns: [

瑞银信
                 交易对手账卡号                对手户名    0
1589    0000300000000224     深圳瑞银信信息技术有限公司客户   14
1590    0000300000000224  深圳瑞银信信息技术有限公司客户备付金  151
1591    0000300000000224         瑞银信支付技术有限公司    2
393901   991451002543849         瑞银信支付技术有限公司   11
393918      991584002549    深圳瑞银信信息技术有限公司备付金    3
393919   991584002549849     深圳瑞银信信息技术有限公司客户  100
联付通
Empty DataFrame
Columns: [交易对手账卡号, 对手户名, 0]
Index: []
合众易宝
Empty DataFrame
Columns: [交易对手账卡号, 对手户名, 0]
Index: []
天下支付
Empty DataFrame
Columns: [交易对手账卡号, 对手户名, 0]
Index: []
商物通
Empty DataFrame
Columns: [交易对手账卡号, 对手户名, 0]
Index: []
汇聚支付
                 交易对手账卡号                对手户名   0
244                           广州市汇聚支付电子科技有限公  85
245                          广州市汇聚支付电子科技有限公司  21
1583    0000300000000165     广州市汇聚支付电子科技有限公司  32
1584    0000300000000165  广州市汇聚支付电子科技有限公司备付金  91
9761           21*****36     广州市汇聚支付电子科技有限公司   1
10225       300000000165  广州市汇聚支付电子科技有限公司备付金   7
393905   991581002598811     广州市汇聚支付电子科技有限公司  16
合利宝
           